# 앙상블 학습의 3형제: 보팅(Voting), 배깅(Bagging), 부스팅(Boosting)

머신러닝에서 **앙상블 학습**은 여러 개의 약한 모델(weak learner)을 모아
더 강력한 예측 모델(strong learner)을 만드는 방법.

---

## 1. 보팅(Voting)
- **여러 다른 모델의 예측 결과를 종합**하는 방법
- 모델이 서로 다른 구조(예: 의사결정나무, 로지스틱 회귀, KNN)일 수 있음
- 투표 방식
  - **Hard Voting**: 각 모델이 낸 예측 클래스에서 다수결로 최종 결정
  - **Soft Voting**: 각 모델이 낸 확률 평균을 계산해 가장 확률이 높은 클래스로 결정

**예시**
> 세 명의 친구에게 날씨를 물어봤을 때, 두 명이 "비 온다"고 하면 비 오는 걸로 결정

<img src='image/voting.png'>
https://bigdaheta.tistory.com/32

---

## 2. 배깅(Bagging, Bootstrap Aggregating)
- **같은 알고리즘**을 여러 번 학습시키되, 각 모델은 **훈련 데이터를 무작위로 복원 추출(bootstrap)** 해서 사용
- 각 모델은 서로 다른 데이터 샘플로 학습 → 예측 결과를 평균(회귀) 또는 투표(분류)
- **대표 알고리즘**: 랜덤포레스트(Random Forest)
- 장점: 분산(variance)을 줄여 과적합 방지

**예시**
> 한 반에서 같은 시험문제를 여러 번 풀게 하되, 매번 다른 학생 그룹을 뽑아 시험을 본 뒤 평균 점수를 내는 것

<img src='image/bagging.png'>
---

## 3. 부스팅(Boosting)
- 모델을 **순차적으로 학습**시키며, 이전 모델이 틀린 데이터에 가중치를 높여 다음 모델이 더 집중하도록 함
- 여러 약한 모델을 차례대로 연결해 강한 모델을 만듦
- **대표 알고리즘**: AdaBoost, Gradient Boosting, XGBoost, LightGBM, CatBoost
- 장점: 편향(bias)과 분산을 모두 줄일 수 있음 (단, 과적합 위험이 있음)

**예시**
> 수학 문제를 풀다가 틀린 문제를 다음 시간에 집중적으로 복습하여 점점 더 실력을 올리는 방식

<img src='image/boosting.png'>
---

## 비교 표

| 구분       | 보팅(Voting)                                | 배깅(Bagging)                              | 부스팅(Boosting)                              |
|------------|---------------------------------------------|---------------------------------------------|-----------------------------------------------|
| 모델 구조  | 서로 다른 모델 가능                         | 같은 모델 여러 개 사용                      | 같은 모델 여러 개 사용                        |
| 학습 방식  | 병렬(parallel)                              | 병렬(parallel)                              | 순차(sequential)                              |
| 데이터 샘플링 | 전체 데이터 사용                           | 복원 추출(bootstrap)                        | 이전 모델의 오차 반영                         |
| 장점       | 간단하고 다양한 모델 결합 가능               | 분산 감소, 과적합 방지                       | 편향·분산 감소, 강력한 성능                     |
| 단점       | 개별 모델이 모두 약하면 효과 적음            | 모델 수 많으면 계산량 증가                    | 과적합 위험, 학습 시간 길어짐                   |
| 대표 알고리즘 | Hard/Soft Voting Classifier               | Random Forest                               | AdaBoost, XGBoost, LightGBM                   |

---





---

## 1. Voting (Hard / Soft)
- **Hard Voting**: 각 모델의 예측 결과를 다수결로 결정
- **Soft Voting**: 각 모델의 예측 확률을 평균하여 최종 클래스 결정


In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.ensemble import VotingClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family']='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

In [10]:
df = pd.read_csv('data/pima-indians-diabetes.data.csv')
x_data = df.iloc[:,:-1].values
y_data = df.iloc[:,-1].values

logistic = LogisticRegression( max_iter=500)
knn = KNeighborsClassifier( n_neighbors=5 )
tree = DecisionTreeClassifier(min_samples_leaf=5)

voting_model = VotingClassifier( [('logistic',logistic),('knn',knn),('tree',tree)],
                                voting='soft')

x_train, x_test, y_train, y_test = train_test_split( x_data,y_data, 
                            test_size=0.2,random_state=42, stratify=y_data)

voting_model.fit( x_train, y_train)

,estimators,"[('logistic', ...), ('knn', ...), ...]"
,voting,'soft'
,weights,None
,n_jobs,None
,flatten_transform,True
,verbose,False
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True


In [11]:
pred = voting_model.predict( x_test)
pred

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [12]:
voting_model.score( x_train, y_train)

0.8729641693811075

In [13]:
voting_model.score( x_test, y_test)

0.7467532467532467

# 랜덤포레스트(Random Forest)는 배깅의 대표적인 예

- **배깅(Bagging)** = Bootstrap Aggregating  
  → 데이터를 복원추출하여 여러 개의 같은 모델을 학습 후 평균/투표  
- **랜덤포레스트(Random Forest)** = 배깅 + 각 노드 분할 시 **특징(변수) 무작위 선택**  
  → 모델 다양성을 더 높여 성능 향상

즉, 랜덤포레스트는 **배깅의 한 종류**입니다.

---


In [16]:
tree = DecisionTreeClassifier()
bagging = BaggingClassifier( estimator=tree, n_estimators=10 )
bagging.fit( x_train, y_train)

,estimator,DecisionTreeClassifier()
,n_estimators,10
,max_samples,1.0
,max_features,1.0
,bootstrap,True
,bootstrap_features,False
,oob_score,False
,warm_start,False
,n_jobs,None
,random_state,None
,verbose,0


In [17]:
bagging.predict(x_test)

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0])

In [18]:
bagging.score( x_test, y_test )

0.7662337662337663

### bagging

In [19]:
rmodel = RandomForestClassifier(n_estimators=100)
rmodel.fit( x_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [20]:
rmodel.score(x_test, y_test)

0.7597402597402597

### boosting


`xgboost`는 **부스팅(Boosting)** 계열의 대표적인 라이브러리입니다.  
`adabag`는 주로 AdaBoost를, `xgboost`는 Gradient Boosting 기반 알고리즘을 구현합니다.

---


In [6]:
%pip install xgboost

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
    --------------------------------------- 1.6/72.0 MB 10.5 MB/s eta 0:00:07
   - -------------------------------------- 2.9/72.0 MB 7.3 MB/s eta 0:00:10
   -- ------------------------------------- 4.5/72.0 MB 7.7 MB/s eta 0:00:09
   --- ------------------------------------ 6.3/72.0 MB 7.7 MB/s eta 0:00:09
   ---- ----------------------------------- 8.1/72.0 MB 8.0 MB/s eta 0:00:08
   ----- ---------------------------------- 9.4/72.0 MB 7.8 MB/s eta 0:00:08
   ------ --------------------------------- 11.5/72.0 MB 8.1 MB/s eta 0:00:08
   ------- -------------------------------- 13.6/72.0 MB 8.3 MB/s eta 0:00:08
   -------- ------------------------------- 15.2/72.0 MB 8.3 MB/s eta 0:00:07
   --------- ------------------------------ 16.8/72.0 MB 8.3 MB/s eta 0:00:07
   ---------- ----------------------------- 18.4/72.0 MB 8.2 MB/s eta 0:00:07
   ----------- ---------------------------- 20.2/72.0 MB 8.1 MB/s eta 0:00:07


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


| 인자명            | 설명                                                                                                                                                                                           |
| -------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **data**       | 학습에 사용할 입력 데이터 (matrix 또는 `xgb.DMatrix` 형식). 여기서는 `iris` 데이터에서 종(Species) 컬럼을 제외한 숫자형 특징들을 사용.                                                                                               |
| **label**      | 각 데이터의 실제 정답(타겟) 벡터. 다중분류의 경우 **0부터 시작하는 정수**로 변환해야 함 (`0, 1, 2` 형태).                                                                                                                        |
| **objective**  | 모델의 학습 목표 함수(목적함수). <br> - `"reg:squarederror"` → 회귀 <br> - `"binary:logistic"` → 이진분류 (확률 출력) <br> - `"multi:softmax"` → 다중분류 (클래스 번호 직접 출력) <br> - `"multi:softprob"` → 다중분류 (각 클래스 확률 출력) |
| **num\_class** | 다중분류 시 클래스 개수 지정. `"multi:softmax"` 또는 `"multi:softprob"`일 때 필수.                                                                                                                             |
| **nrounds**    | 부스팅 반복 횟수(트리 개수). 값이 크면 더 학습하지만 과적합 위험이 있음.                                                                                                                                                  |
| **verbose**    | 학습 중 출력 여부 (`0`이면 조용히, `1`이면 진행 상황 출력).                                                                                                                                                      |
